In [1]:
try:
    output_filepath
except:
    output_filepath = '../data/processed/'

In [2]:
import os
from pathlib import Path

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [30]:
gdp = pd.read_parquet(output_filepath + '2015/industry_output.parquet')
len(gdp)

190

In [3]:
from src.data.migration_network import MigrationNetworkCreation

MNC = MigrationNetworkCreation(
        year='2010', input_filepath='../data/raw/', output_filepath=output_filepath
    )
MNC.run()

In [4]:
len(MNC.G)

180

In [13]:
MNC.G.in_edges(nbunch=['CHN'], data=True)

InEdgeDataView([])

In [8]:
import numpy as np
np.sqrt(len(MNC.G.edges(data=True)))

58.18934610390462

In [27]:
len(set(MNC.G.nodes()).intersection(set(gdp.index)))

180

In [45]:
from src.data.financial_network import IndustryNetworkCreationEORA

INC = IndustryNetworkCreationEORA(
            year='2000', input_filepath='../data/raw/', output_filepath=output_filepath
        )
INC.run()

In [9]:
len(INC.node_index)

NameError: name 'INC' is not defined

In [15]:
data_path = os.path.join('../data/raw/', 'API_SL.TLF.TOTL.IN_DS2_en_csv_v2_1929128.csv')
df_population = pd.read_csv(data_path, skiprows=4)
df_population.drop(columns=['Country Name','Indicator Name', 'Indicator Code'], inplace=True)

df_population = df_population.set_index(['Country Code']).stack().reset_index()

df_population.columns = ['country', 'year','wkn_population']

df_population['log_wkn_population'] = df_population['wkn_population'].map(lambda x: np.log(x + 1))
df_population.country.nunique()

233